In [1]:
using Plots;plotly()
using Optim
using Zygote
using Meteor.Utility
using Meteor.ExactDiag
using DataFrames;using CSV
using Pkg

┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots /home/nudt/.julia/packages/Plots/kyYZF/src/backends.jl:372


In [2]:
using LinearAlgebra
ccall((:openblas_get_num_threads64_, Base.libblas_name),Cint,())
LinearAlgebra.BLAS.set_num_threads(8)

In [3]:
using Base.Threads;nthreads()

50

In [4]:
function thermal_state(dims,nbar)
    c::Array{Complex{Float64}}=zeros(dims,dims)
    if nbar==0
        c[1,1]=1
    else
        for i in 1:dims
           c[i,i]= exp(-i*log(1/nbar+1))
        end
    end
    c=c/tr(c)
    return c        
end

function fock_state(dims,n)
    c::Matrix{ComplexF64}=zeros(dims,dims)
    if n<dims
        c[n+1,n+1]=1
    end
    return c
end

function eye(n)
    k::Array{Complex{Float64}}=zeros(n,n)
    for i in 1:n
        k[i,i]=1
    end
    return k    
end


function a₋(n)      #n为维数
    s::Array{Complex{Float64}}=zeros(n,n)
    for i=1:n-1
        s[i,i+1]=sqrt(i)
    end
    return s
end

a₊(n)=adjoint(a₋(n));

dim=10
ν=1
dot_num=30
initial_state=kron([1.0 0 0;0 0 0;0 0 0],thermal_state(dim,1))
#########密度矩阵向量化，超算子维数变成n^2*n^2维,态矩阵厄米，只用存一半？意义不大
function dm2vec(dm)#态向量化
#julia里是按列reshape的,reshape成列向量
    return reshape(dm,:,1)
end

function vec2dm(vec)
    return reshape(vec,3*dim,:)
end


n=kron(eye(3),a₊(dim)*a₋(dim))
en=kron([0 0 0;0 0 0;0 0 1],eye(dim))
rr=kron([0 0 0;0 1 0;0 0 0],eye(dim))
η1=-0.15
η2=0.15
a=a₊(dim)+a₋(dim)
ag2=kron([0 0 0;0 0 0;1 0 0],exp(im*η1*a))
ag3=kron([0 0 1;0 0 0;0 0 0],exp(-im*η1*a))
ag4=ag2+ag3
ar2=kron([0 0 0;0 0 0;0 1 0],exp(im*η2*a))
ar3=kron([0 0 0;0 0 1;0 0 0],exp(-im*η2*a))
ar4=ar2+ar3

E=eye(3*dim)
γ1=20/3*ν
γ2=40/3*ν

function diss1()
    dtheta = 0.01
    x=zeros((3*dim)^2,(3*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*η1*theta*a)
        aup=kron([0 0 0;0 0 0;1 0 0],jump')
        adown=kron([0 0 1;0 0 0;0 0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end
function diss2()
    dtheta = 0.01
    x=zeros((3*dim)^2,(3*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*η2*theta*a)
        aup=kron([0 0 0;0 0 0;0 1 0],jump')
        adown=kron([0 0 0;0 0 1;0 0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end

dissipation1=γ1*diss1()
dissipation2=γ2*diss2()

# Δg=(Ωr^2+Ωg^2)/ν-ν
# Δr=Δg

function eit_Lindbladian_vec(Ωg,Ωr,Δ)
    unitary::Matrix{ComplexF64}=kron(E,-im*(ν*n-Δ*en+Ωg/2*ag4+Ωr/2*ar4))+kron(transpose(im*((ν*n-Δ*en+Ωg/2*ag4+Ωr/2*ar4))),E)
    
    return unitary+dissipation1+dissipation2
end

instate=dm2vec(initial_state);


In [6]:
LinearAlgebra.BLAS.set_num_threads(1)
function nt()
    ttot=200
    space=0.2
    s_omega_g=0.2:space: 20.2
    s_omega_r=0.2:space: 20.2
    nf = zeros(length(s_omega_g),length(s_omega_r))
    datane=DataFrame(omega_g=s_omega_g,omega_r=s_omega_r)
    @threads for omegag in s_omega_g
        for omegar in s_omega_r
            nf[convert(Int64,round((omegag-s_omega_g[1])/space+1)),convert(Int64,round((omegar-s_omega_r[1])/space+1))]=real(tr(n*vec2dm(exp(ttot*eit_Lindbladian_vec(omegag,omegar,65))*instate)))
        end
        print(omegag," ")
    end
    datane=hcat(datane,DataFrame(nf,:auto))
    CSV.write("omegag_omegar_sweep65.csv", datane)
    return nf
end
nt()

9.2 2.4 0.2 8.0 14.0 12.0 8.4 1.2 11.6 7.6 11.2 15.6 17.2 18.0 7.2 17.6 19.6 2.8 6.8 0.8 18.4 6.4 5.2 3.2 20.0 4.8 8.8 4.0 5.6 2.0 15.2 3.6 19.2 16.4 10.4 12.4 10.8 16.0 4.4 12.8 16.8 10.0 1.6 14.8 9.6 14.4 13.2 6.0 13.6 18.8 17.8 17.6 0.4 2.6 2.4 8.2 8.0 12.2 12.0 8.6 8.4 14.2 14.0 9.4 9.2 17.4 17.2 11.4 11.2 19.8 19.6 9.0 8.8 15.8 15.6 7.8 7.6 11.8 11.6 7.0 6.8 1.4 1.2 18.6 18.4 4.2 4.0 11.0 10.8 18.2 18.0 5.0 4.8 7.4 7.2 15.4 15.2 10.6 10.4 16.6 16.4 1.0 0.8 6.6 6.4 3.0 2.8 12.6 12.4 20.2 20.0 10.2 10.0 5.4 5.2 3.4 3.2 2.2 2.0 5.8 5.6 19.4 19.2 16.2 16.0 17.0 16.8 9.8 9.6 3.8 3.6 13.8 13.6 13.0 12.8 15.0 14.8 4.6 4.4 13.4 13.2 1.8 1.6 14.6 14.4 19.0 18.8 6.2 6.0 12.2 17.8 8.2 8.6 2.6 0.6 19.8 7.8 14.2 9.4 

101×101 Matrix{Float64}:
 0.990535  0.99054   0.990552  0.990575  …  0.987388  0.987576  0.987725
 0.991454  0.991469  0.991509  0.991595     0.978967  0.979707  0.980296
 0.992945  0.992965  0.993032  0.993192     0.965228  0.966853  0.968148
 0.994947  0.994951  0.995018  0.995229     0.946591  0.949385  0.951616
 0.997384  0.997334  0.997344  0.99754      0.923604  0.927788  0.931136
 1.00017   1.00001   0.999888  0.99997   …  0.896906  0.902632  0.907226
 1.00321   1.00289   1.00255   1.00241      0.86719   0.874537  0.88045
 1.00641   1.0059    1.00527   1.00482      0.835168  0.844147  0.851398
 1.0097    1.00899   1.00807   1.0073       0.801537  0.812098  0.820655
 1.013     1.01214   1.011     1.00997      0.766954  0.778992  0.788784
 1.01626   1.01535   1.01412   1.01299   …  0.732014  0.745386  0.756306
 1.01941   1.01857   1.01744   1.01641      0.69724   0.711774  0.723689
 1.02241   1.02174   1.02086   1.02009      0.663074  0.67858   0.691342
 ⋮                         

In [7]:
data3d=CSV.read("omegag_omegar_sweep65.csv", DataFrame)
nsurf=Array(data3d[:,Between(:x1, end)])
surface(data3d.omega_g,data3d.omega_r,nsurf,xlabel="omega_g",ylabel="omega_r",zlabel="n_t",xticks=0:2:26,yticks=0:2:26)

15.4

<!DOCTYPE html>
 
 
 Plots.jl

 1.0 4.2 10.6 6.6 15.8 18.2 19.0 16.6 11.0 5.0 11.8 3.4 9.0 7.4 4.6 11.4 18.6 17.4 12.6 7.0 10.2 16.2 1.4 15.0 3.0 13.8 5.4 20.2 13.4 2.2 19.4 17.0 1.8 3.8 5.8 9.8 13.0 14.6 6.2 